# 原型模式 (Prototype Pattern) 实战演示

## 概述

原型模式是一种创建型设计模式，它允许通过复制现有对象来创建新对象，而不需要依赖于它们的具体类。

本Notebook将通过可执行的代码演示：
- 浅拷贝 vs 深拷贝的区别
- 原型模式的实际应用
- 测试用例的验证

## 核心概念

- **浅拷贝 (Shallow Copy)**: 创建新对象实例，但引用类型字段仍然指向原始对象的相同实例
- **深拷贝 (Deep Clone)**: 创建完全独立的副本，递归复制所有引用类型字段

In [ ]:
// 首先定义原型接口
public interface IPrototype<T>
{
    /// <summary>
    /// 浅拷贝 - 创建当前对象的浅表副本。
    /// 浅拷贝会复制值类型字段，但引用类型字段仍然指向原始对象。
    /// </summary>
    T ShallowCopy();

    /// <summary>
    /// 深拷贝 - 创建当前对象的完全独立副本。
    /// 深拷贝会递归复制所有字段，确保克隆体与原始对象没有任何共享引用。
    /// </summary>
    T DeepClone();
}

Console.WriteLine("✅ 原型接口定义完成");

In [ ]:
// 定义嵌套对象
public class NestedObject : IPrototype<NestedObject>
{
    public string Data { get; set; } = "Default Data";
    public int Counter { get; set; } = 0;

    public NestedObject ShallowCopy()
    {
        return new NestedObject
        {
            Data = this.Data,
            Counter = this.Counter
        };
    }

    public NestedObject DeepClone()
    {
        return ShallowCopy(); // 对于简单对象，浅拷贝等同于深拷贝
    }

    public override string ToString()
    {
        return $"NestedObject(Data={Data}, Counter={Counter})";
    }
}

System.Console.WriteLine("✅ 嵌套对象类定义完成");

In [ ]:
// 定义主原型类
public class Prototype : IPrototype<Prototype>
{
    public string Name { get; set; }
    public List<string> Items { get; set; } = new List<string>();
    public Dictionary<string, object> Properties { get; set; } = new Dictionary<string, object>();
    public NestedObject Nested { get; set; } = new NestedObject();

    public Prototype(string name)
    {
        Name = name;
    }

    // 浅拷贝 - 引用类型字段仍然指向原始对象
    public Prototype ShallowCopy()
    {
        return (Prototype)this.MemberwiseClone();
    }

    // 深拷贝 - 完全独立的副本
    public Prototype DeepClone()
    {
        Prototype clone = new Prototype(Name)
        {
            // 深拷贝集合
            Items = new List<string>(this.Items),

            // 深拷贝字典
            Properties = new Dictionary<string, object>(this.Properties),

            // 深拷贝嵌套对象
            Nested = this.Nested.DeepClone()
        };

        return clone;
    }

    public override string ToString()
    {
        string itemsStr = string.Join(", ", Items);
        string propsStr = string.Join(", ", Properties.Select(p => $"{p.Key}={p.Value}"));
        return $"Prototype: Name={Name}, Items=[{itemsStr}], Properties=[{propsStr}], Nested={Nested}";
    }
}

Console.WriteLine("✅ 主原型类定义完成");

## 🧪 浅拷贝演示

让我们创建一个原型对象并演示浅拷贝的行为：

In [ ]:
// 创建原始对象并添加数据
var original = new Prototype("测试对象");
original.Items.Add("item1");
original.Items.Add("item2");
original.Properties["key1"] = "value1";
original.Nested.Data = "原始数据";
original.Nested.Counter = 42;

Console.WriteLine("📋 原始对象:");
Console.WriteLine(original);
Console.WriteLine();

In [ ]:
// 执行浅拷贝
var shallowClone = original.ShallowCopy();

Console.WriteLine("🔄 浅拷贝结果:");
Console.WriteLine($"原始对象: {original}");
Console.WriteLine($"浅拷贝对象: {shallowClone}");
Console.WriteLine();

// 验证浅拷贝特性
Console.WriteLine("🔍 浅拷贝验证:");
Console.WriteLine($"对象实例相同吗? {object.ReferenceEquals(original, shallowClone)}");
Console.WriteLine($"Items集合相同吗? {object.ReferenceEquals(original.Items, shallowClone.Items)}");
Console.WriteLine($"Properties字典相同吗? {object.ReferenceEquals(original.Properties, shallowClone.Properties)}");
Console.WriteLine($"嵌套对象相同吗? {object.ReferenceEquals(original.Nested, shallowClone.Nested)}");
Console.WriteLine();

In [ ]:
// 演示浅拷贝的共享引用问题
Console.WriteLine("⚠️ 浅拷贝的共享引用演示:");

// 修改原始对象的集合
original.Items.Add("新增项目");
original.Properties["newKey"] = "新值";
original.Nested.Data = "修改后的数据";

Console.WriteLine("修改原始对象后:");
Console.WriteLine($"原始对象: {original}");
Console.WriteLine($"浅拷贝对象: {shallowClone}");
Console.WriteLine();
Console.WriteLine("👀 注意：浅拷贝对象的引用类型字段也被修改了！");

## 🔬 深拷贝演示

现在让我们演示深拷贝，它会创建完全独立的副本：

In [ ]:
// 创建新的原始对象用于深拷贝演示
var original2 = new Prototype("深拷贝测试");
original2.Items.Add("深拷贝项目1");
original2.Items.Add("深拷贝项目2");
original2.Properties["deepKey"] = "深拷贝值";
original2.Nested.Data = "深拷贝数据";
original2.Nested.Counter = 100;

Console.WriteLine("📋 原始对象 (用于深拷贝):");
Console.WriteLine(original2);
Console.WriteLine();

In [ ]:
// 执行深拷贝
var deepClone = original2.DeepClone();

Console.WriteLine("🔄 深拷贝结果:");
Console.WriteLine($"原始对象: {original2}");
Console.WriteLine($"深拷贝对象: {deepClone}");
Console.WriteLine();

// 验证深拷贝特性
Console.WriteLine("🔍 深拷贝验证:");
Console.WriteLine($"对象实例相同吗? {object.ReferenceEquals(original2, deepClone)}");
Console.WriteLine($"Items集合相同吗? {object.ReferenceEquals(original2.Items, deepClone.Items)}");
Console.WriteLine($"Properties字典相同吗? {object.ReferenceEquals(original2.Properties, deepClone.Properties)}");
Console.WriteLine($"嵌套对象相同吗? {object.ReferenceEquals(original2.Nested, deepClone.Nested)}");
Console.WriteLine();

// 验证内容是否相等
Console.WriteLine("📊 内容比较:");
Console.WriteLine($"Items内容相等吗? {original2.Items.SequenceEqual(deepClone.Items)}");
Console.WriteLine($"嵌套对象数据相等吗? {original2.Nested.Data == deepClone.Nested.Data}");
Console.WriteLine($"嵌套对象计数器相等吗? {original2.Nested.Counter == deepClone.Nested.Counter}");

In [ ]:
// 演示深拷贝的独立性
Console.WriteLine();
Console.WriteLine("✅ 深拷贝的独立性演示:");

// 修改原始对象
original2.Items.Add("修改后的项目");
original2.Properties["modifiedKey"] = "修改后的值";
original2.Nested.Data = "修改后的嵌套数据";
original2.Nested.Counter = 999;

Console.WriteLine("修改原始对象后:");
Console.WriteLine($"原始对象: {original2}");
Console.WriteLine($"深拷贝对象: {deepClone}");
Console.WriteLine();
Console.WriteLine("🎉 注意：深拷贝对象保持独立，没有受到影响！");

## 🎯 实际应用场景

让我们看看原型模式在实际开发中的应用：

In [ ]:
// 游戏角色模板示例
public class GameCharacter : IPrototype<GameCharacter>
{
    public string Name { get; set; }
    public int Health { get; set; }
    public int Attack { get; set; }
    public List<string> Skills { get; set; } = new List<string>();
    
    public GameCharacter(string name, int health, int attack)
    {
        Name = name;
        Health = health;
        Attack = attack;
    }
    
    public GameCharacter ShallowCopy()
    {
        return (GameCharacter)this.MemberwiseClone();
    }
    
    public GameCharacter DeepClone()
    {
        return new GameCharacter(Name, Health, Attack)
        {
            Skills = new List<string>(Skills)
        };
    }
    
    public override string ToString()
    {
        return $"角色: {Name} (生命值:{Health}, 攻击力:{Attack}, 技能:[{string.Join(", ", Skills)}])";
    }
}

// 创建角色模板
var warriorTemplate = new GameCharacter("战士模板", 100, 50);
warriorTemplate.Skills.Add("剑击");
warriorTemplate.Skills.Add("盾击");

Console.WriteLine("🎮 游戏角色模板:");
Console.WriteLine(warriorTemplate);

In [ ]:
// 快速创建多个角色实例
var warrior1 = warriorTemplate.DeepClone();
warrior1.Name = "战士A";

var warrior2 = warriorTemplate.DeepClone();
warrior2.Name = "战士B";
warrior2.Health = 120; // 个性化属性

var warrior3 = warriorTemplate.DeepClone();
warrior3.Name = "战士C";
warrior3.Skills.Add("旋风斩"); // 额外技能

Console.WriteLine();
Console.WriteLine("⚔️ 基于模板创建的角色:");
Console.WriteLine(warrior1);
Console.WriteLine(warrior2);
Console.WriteLine(warrior3);

Console.WriteLine();
Console.WriteLine("📈 性能优势: 无需重复设置基础属性和技能！");

## 📊 性能对比

让我们比较一下使用原型模式与直接创建对象的性能差异：

In [ ]:
using System.Diagnostics;

// 复杂对象模拟创建成本
public class ComplexObject : IPrototype<ComplexObject>
{
    public string Data { get; set; }
    public List<int> Numbers { get; set; } = new List<int>();
    
    public ComplexObject()
    {
        // 模拟昂贵的初始化过程
        Data = "复杂数据" + DateTime.Now.Ticks;
        for (int i = 0; i < 1000; i++)
        {
            Numbers.Add(i * i);
        }
    }
    
    public ComplexObject ShallowCopy()
    {
        return (ComplexObject)MemberwiseClone();
    }
    
    public ComplexObject DeepClone()
    {
        return new ComplexObject 
        { 
            Data = this.Data,
            Numbers = new List<int>(this.Numbers)
        };
    }
}

Console.WriteLine("✅ 复杂对象类定义完成");

In [ ]:
// 性能测试
const int testCount = 100;

// 1. 直接创建对象
var sw1 = Stopwatch.StartNew();
var directObjects = new List<ComplexObject>();
for (int i = 0; i < testCount; i++)
{
    directObjects.Add(new ComplexObject());
}
sw1.Stop();

// 2. 使用原型模式
var prototype = new ComplexObject();
var sw2 = Stopwatch.StartNew();
var clonedObjects = new List<ComplexObject>();
for (int i = 0; i < testCount; i++)
{
    clonedObjects.Add(prototype.DeepClone());
}
sw2.Stop();

Console.WriteLine("⚡ 性能测试结果:");
Console.WriteLine($"直接创建 {testCount} 个对象耗时: {sw1.ElapsedMilliseconds} ms");
Console.WriteLine($"使用深拷贝创建 {testCount} 个对象耗时: {sw2.ElapsedMilliseconds} ms");
Console.WriteLine($"性能提升: {(double)sw1.ElapsedMilliseconds / sw2.ElapsedMilliseconds:F2}x");

Console.WriteLine();
Console.WriteLine("🚀 原型模式特别适合创建成本高昂的对象！");

## 📝 总结

### 🎯 原型模式的关键点

1. **浅拷贝 vs 深拷贝**：
   - 浅拷贝：引用类型字段共享
   - 深拷贝：完全独立的副本

2. **适用场景**：
   - 对象创建成本高昂
   - 需要创建大量相似对象
   - 避免复杂的初始化过程

3. **实现要点**：
   - `MemberwiseClone()` 实现浅拷贝
   - 手动复制引用类型实现深拷贝
   - 考虑循环引用问题

### ✅ 优势
- 性能优化（避免重复初始化）
- 代码简洁（复用现有对象）
- 灵活性高（运行时确定类型）

### ⚠️ 注意事项
- 深拷贝可能消耗更多资源
- 需要正确处理引用类型
- 循环引用需要特殊处理